In [ ]:
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt
from google.colab import drive
import pickle
!pip install evaluate
import evaluate

# --- 1. Mount Google Drive and Load Data ---
drive.mount('/content/drive')

# Download NLTK data
nltk.download('vader_lexicon')
nltk.download('punkt')

# File path
file_path = './amazon_reviews_us_Watches_v1_00.tsv'

print("Loading and Preprocessing Data...")

# Load TSV file
data = pd.read_csv(file_path, sep='\t', on_bad_lines='skip', quoting=3)

# Handle missing values
data['review_body'] = data['review_body'].fillna('')

# Create sentiment labels based on star rating
data['sentiment'] = data['star_rating'].apply(lambda x: 'positive' if x >= 4 else 'negative')

# Reduce dataset for testing
data = data.sample(50000, random_state=42)  # Use 50,000 reviews for faster processing

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data['review_body'], data['sentiment'], test_size=0.2, random_state=42
)

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

print(f"Number of training samples: {len(X_train)}")
print(f"Number of testing samples: {len(X_test)}")

# --- 2. Tokenize Data ---
print("Tokenizing Data...")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize in chunks to save memory
def tokenize_data_in_chunks(data, chunk_size=5000):
    encodings = {"input_ids": [], "attention_mask": []}
    for i in tqdm(range(0, len(data), chunk_size), desc="Tokenizing Data in Chunks"):
        batch = data[i:i + chunk_size]
        tokenized = tokenizer(list(batch), truncation=True, padding=True, max_length=512)
        encodings["input_ids"].extend(tokenized["input_ids"])
        encodings["attention_mask"].extend(tokenized["attention_mask"])
    return encodings

train_encodings = tokenize_data_in_chunks(X_train, chunk_size=5000)
test_encodings = tokenize_data_in_chunks(X_test, chunk_size=2000)

# Save tokenized data
with open('train_encodings.pkl', 'wb') as f:
    pickle.dump(train_encodings, f)
with open('test_encodings.pkl', 'wb') as f:
    pickle.dump(test_encodings, f)

# --- 3. Create Dataset Class ---
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, y_train_encoded)
test_dataset = SentimentDataset(test_encodings, y_test_encoded)

# --- 4. Load Pre-trained Model ---
print("Loading Pre-trained Model...")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# --- 5. Define Training Arguments ---
print("Setting Training Arguments...")
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True
)

# --- 6. Train the Model ---
print("Training the Model...")
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = evaluate.load("accuracy").compute(predictions=preds, references=labels)["accuracy"]  
    return {"eval_accuracy": accuracy}

# Pass compute_metrics to Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics # Add this line
)


trainer.train()

# --- 7. Evaluate Hugging Face Model ---
print("Evaluating Hugging Face Model...")
metrics = trainer.evaluate()
print("Hugging Face Evaluation Metrics:")
print(metrics)

predictions = trainer.predict(test_dataset)
predicted_labels = predictions.predictions.argmax(axis=1)
predicted_labels_decoded = label_encoder.inverse_transform(predicted_labels)

# Classification Report
hugging_face_report = classification_report(y_test, predicted_labels_decoded, output_dict=True)
print("Hugging Face Classification Report:")
print(classification_report(y_test, predicted_labels_decoded))

# --- 8. Logistic Regression Comparison ---
print("Training Logistic Regression for Comparison...")
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=10000, ngram_range=(1, 2), stop_words='english')
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

logistic_model = LogisticRegression(class_weight='balanced', solver='liblinear')
logistic_model.fit(X_train_tfidf, y_train)

y_pred_logistic = logistic_model.predict(X_test_tfidf)

logistic_report = classification_report(y_test, y_pred_logistic, output_dict=True)
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_logistic))

# --- 9. Visualize Comparison ---
print("Visualizing Comparison...")

hugging_face_metrics = [
    hugging_face_report['accuracy'],
    hugging_face_report['macro avg']['precision'],
    hugging_face_report['macro avg']['recall'],
    hugging_face_report['macro avg']['f1-score'],
]

logistic_metrics = [
    accuracy_score(y_test, y_pred_logistic),
    precision_score(y_test, y_pred_logistic, average='macro'),
    recall_score(y_test, y_pred_logistic, average='macro'),
    f1_score(y_test, y_pred_logistic, average='macro'),
]

metric_labels = ['Accuracy', 'Precision', 'Recall', 'F1-Score']

# Plot metrics
x = np.arange(len(metric_labels))
width = 0.35

plt.figure(figsize=(10, 6))
plt.bar(x - width/2, logistic_metrics, width, label='Logistic Regression')
plt.bar(x + width/2, hugging_face_metrics, width, label='Hugging Face')

plt.xlabel('Metrics')
plt.ylabel('Scores')
plt.title('Comparison of Logistic Regression and Hugging Face Models')
plt.xticks(x, metric_labels)
plt.legend()
plt.show()
